<a href="https://colab.research.google.com/github/TKph/colab/blob/main/NaiveBayes%E3%81%AB%E3%82%88%E3%82%8B%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---

# データの読み込み

---



In [2]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint

train_set = fetch_20newsgroups(subset = 'train', random_state = 42)
test_set = fetch_20newsgroups(subset = 'test', random_state = 42)

X_train = train_set.data
y_train = train_set.target
X_test = test_set.data
y_test = test_set.target



---

# 事前準備

---



In [ ]:
print('カテゴリ一覧')
pprint(train_set.target_names)
print('\n')
print(f'News Script: \n {X_train[0]}')
print('記事その１のカテゴリ')
print(f'Text Category label: {y_train[0]}')

Bag-of-Word表現の実装

BoW表現では
'This is an apple. I eat this apple' は (This, is , an , apple, I eat) = (2, 1, 1, 2, 1, 1)と表現される. またカテゴリに関係なく出現する単語(this,that,it,a,an,I,be動詞など)をstop wordとして登録しておいてそれらを除くと, 先の文は (apple, eat) = (2, 1)と表現される.

単語をベクトルに変換する方法は他にも色々あり, 知りたい場合はscikit公式ドキュメントを参照する事.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words = 'english')
vectorizer.fit(X_train)
X_train_bow = vectorizer.transform(X_train)
X_test_bow = vectorizer.transform(X_test)

print('(テキスト番号, 単語番号) 出現回数')
print(X_train_bow[0])
print('\nBoW表現ベクトル')
print(X_train_bow[0].toarray())



---
# NaiveBayesの実装


---



scikitのnaive_bayesにはBernoulliNB(離散データの2クラス分類)やMultinomialNB(離散データの多クラス分類), GaussianNB(連続データのクラス分類)があるが, 今回は離散データを20種類に分けるのでMultinomialNBを使う.

In [6]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha = 0.4)  #alphaのデフォルトは1
mnb.fit(X_train_bow, y_train)

print(f'Train Accuracy: {mnb.score(X_train_bow, y_train):.3f}')
print(f'Test Accuracy: {mnb.score(X_test_bow, y_test):.3f}')

Train Accuracy: 0.951
Test Accuracy: 0.811


In [7]:
mnb_small = MultinomialNB(alpha = 0.001)
mnb_small.fit(X_train_bow, y_train)

print(f'Train Accuracy(small): {mnb_small.score(X_train_bow, y_train):.3f}')
print(f'Test Accuracy(small): {mnb_small.score(X_test_bow, y_test):.3f}')

mnb_large = MultinomialNB(alpha = 100)
mnb_large.fit(X_train_bow, y_train)

print(f'Train Accuracy(large): {mnb_large.score(X_train_bow, y_train):.3f}')
print(f'Test Accuracy(large): {mnb_large.score(X_test_bow, y_test):.3f}')

Train Accuracy(small): 0.988
Test Accuracy(small): 0.799
Train Accuracy(large): 0.747
Test Accuracy(large): 0.632




---

# 解説

---



みんな大好きベイズの定理を使う.

$$ p(\theta|D)=\frac{p(D|\theta)p(\theta)}{p(D)} $$

今回は, $D$がテキスト, $\theta$はカテゴリである. 目的は左辺を求めることであるが, それは難しいので右辺から求める. 
改めてcategoryとtextを使って先の定理を分かりやすくしておこう.

$$ p(Category|text) = \frac{p(Text|Category)p(Category)}{p(Text)} \quad -①$$

それぞれの確率の求め方を説明していく. 
$$ p(category = c) = \frac{カテゴリcのテキスト数}{全テキスト数} $$

$$ p(Text|Category) = p(word_1, word_2, ..., word_K|Category) $$

ここでそれぞれの$word_i$は独立に発生すると仮定する. 本当はそんなことはなく, 'サッカー'と'ゴール'は共起しやすいが, 計算が複雑になるのでしょうがない. 右辺の$word_1, word_2, ..., word_K$は同時確率であるが, ここで互いに独立なので積に展開できる. つまり,

$$ p(Text|Category) = \prod_i p(word_i|Category)$$

カテゴリを固定すれば,

$$ p(word_i|Category=c) = \frac{カテゴリcに属するテキスト中に存在するword_iの出現回数}{カテゴリcに属する全単語の出現回数}$$

例を見ていこう. 


1.   テキスト1 | BoW表現 = [0,0,0,...,1]  |カテゴリ7
2.   テキスト2 | BoW表現 = [0,0,0,...,0]  |カテゴリ13
1.   テキスト3 | BoW表現 = [0,1,1,...,0]  |カテゴリ2
2.   テキスト4 | BoW表現 = [1,0,1,...,0]  |カテゴリ3
1.   テキスト5 | BoW表現 = [0,0,0,...,0]  |カテゴリ6
2.   テキスト6 | BoW表現 = [0,0,1,...,1]  |カテゴリ3
                :
                :
99.   テキスト99 | BoW表現 = [2,0,0,...,0]  |カテゴリ10
100.   テキスト100 | BoW表現 = [0,0,0,...,2] | カテゴリ3

ここでカテゴリ3に属するテキストが4,6,100番だけだとする. このときそれぞれの確率は

$$ p(category=3) = \frac{カテゴリ3のテキスト数}{全テキスト数} = \frac{3}{100}$$
$$ p(word_3|Category=3) = \frac{カテゴリ3に属するテキスト中に存在するword_3の出現回数}{カテゴリ3に属する全単語の出現回数} = \frac{2}{6}$$

これを$word_1,word_2,...,word_K$について計算し, 積をとれば$p(Text|Category)$が求まる. このようにして式①を求める. 今はカテゴリ3について計算したが, それをカテゴリ1, カテゴリ2についても計算しそのうち最大のものが分類結果となる. 


---


ここで一つ問題が生じる. 訓練データにない単語がテストデータに出てきたときの挙動を考えてみると, $ p(word_n|Category=c) =0 $になってしまう. そうすると, 積で表される$p(Text|Category=c)$も0になる. つまり新しい単語が存在するだけで、すべてのカテゴリに属する確率が0になってしまい分類ができない(ゼロ頻度問題). これを回避するため, スムージングという手法を用いる. $ p(word_n|Category=c) =0 $とならないように, 計算方法を新しく定義する.

$$ p(word_i|Category=c) = \frac{カテゴリcに属するテキスト中に存在するword_iの出現回数 + \alpha}{カテゴリcに属する全単語の出現回数 + \alpha V}$$
ここで$V$は全単語の種類数である. 先の例を使うとBoW表現の次元数である. 

このようにすれば新しい単語が出てきても, つまりカテゴリcに属するテキスト中に存在する$word_i$の出現回数が0でも, $\alpha$が加算されているので$p(word|category)$は0にはならない. 特に$\alpha=1$の場合をラプラススムージングという. 実装時に引数として設定したalphaがこの$\alpha$に相当する. alphaが小さいと新しい単語が出てきたときの$p(word|category)$が0に近づき訓練データにしか対応できなくなってしまう. 逆にalphaが大きすぎると, カテゴリcに属するテキスト中に存在するword_iの出現回数の影響力が弱くなり, 識別能が低くなる.


